In [0]:
# Read
dfCliente = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/tacnampt@gmail.com/cliente.csv")
dfVenta = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/shared_uploads/tacnampt@gmail.com/venta.csv")

In [0]:
dfCliente.printSchema()
dfCliente.show(2)

root
 |-- IdCliente: string (nullable = true)
 |-- Nombre_y_Apellido: string (nullable = true)
 |-- Domicilio: string (nullable = true)
 |-- Telefono: string (nullable = true)
 |-- Edad: string (nullable = true)
 |-- Rango_Etario: string (nullable = true)
 |-- IdLocalidad: string (nullable = true)
 |-- Latitud: string (nullable = true)
 |-- Longitud: string (nullable = true)

+---------+------------------+--------------------+--------+----+-----------------+-----------+------------+------------+
|IdCliente| Nombre_y_Apellido|           Domicilio|Telefono|Edad|     Rango_Etario|IdLocalidad|     Latitud|    Longitud|
+---------+------------------+--------------------+--------+----+-----------------+-----------+------------+------------+
|        1|Heber Joni Santana|Las Heras Y Bat. ...| 42-5161|  58|4_De 51 a 60 a�os|        189|-34.30997088|-58.81850307|
|        2|       Ana Sapriza|Pueyrredon Y Dupu...| 49-7578|  61|  5_Desde 60 a�os|        264|-34.93908311|-58.73073751|
+---------+

In [0]:
dfVenta.printSchema()
dfVenta.show()

root
 |-- IdVenta: string (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- Fecha_Entrega: string (nullable = true)
 |-- IdCanal: string (nullable = true)
 |-- IdCliente: integer (nullable = true)
 |-- IdSucursal: string (nullable = true)
 |-- IdEmpleado: string (nullable = true)
 |-- IdProducto: string (nullable = true)
 |-- Precio: decimal(10,2) (nullable = true)
 |-- Cantidad: integer (nullable = true)

+-------+----------+-------------+-------+---------+----------+----------+----------+-------+--------+
|IdVenta|     Fecha|Fecha_Entrega|IdCanal|IdCliente|IdSucursal|IdEmpleado|IdProducto| Precio|Cantidad|
+-------+----------+-------------+-------+---------+----------+----------+----------+-------+--------+
|     10|2019-03-16|   2019-03-17|      1|     1003|        13|  13001674|     42894| 515.00|       2|
|     17|2019-02-07|   2019-02-08|      2|     1590|        13|  13001674|     42976| 216.00|       1|
|     18|2019-02-14|   2019-02-22|      2|     3029|        13|  13

In [0]:
# CAMBIANDO TIPO DATO DE LAS COLUMNAS

from pyspark.sql import functions as F
from pyspark.sql.types import *

# DATAFRAME CLIENTE
dfCliente = dfCliente.withColumn("IdCliente", dfCliente["IdCliente"].cast(IntegerType()))

# DATAFRAME VENTA
dfVenta = dfVenta.withColumn("IdCliente", dfVenta["IdCliente"].cast(IntegerType()))
dfVenta = dfVenta.withColumn("Precio", dfVenta["Precio"].cast(DecimalType(10,2)))
dfVenta = dfVenta.withColumn("Cantidad", dfVenta["Cantidad"].cast(IntegerType()))
dfVenta = dfVenta.withColumn("Fecha", dfVenta["Fecha"].cast(DateType()))

In [0]:
# CREANDO VISTA POR CADA DATAFRAME
dfCliente.createOrReplaceTempView("cliente")
dfVenta.createOrReplaceTempView("venta")

In [0]:
# USANDO spar.sql PARA HACER CONSULTAS SQL
spark.sql(""" 
SELECT c.Nombre_y_Apellido, SUM(v.Precio * v.Cantidad) as total
FROM venta v
INNER JOIN cliente c ON v.IdCliente = c.IdCliente
GROUP BY c.Nombre_y_Apellido
ORDER BY total DESC
LIMIT 5
           """).show(truncate=False)

+-----------------+---------+
|Nombre_y_Apellido|total    |
+-----------------+---------+
|Laura Ines Guasch|696645.52|
|Sergio Artazu    |695813.00|
|Antonio Benitez  |621303.84|
|Irma Esquivel    |546859.00|
|Sin Dato         |541618.00|
+-----------------+---------+



In [0]:
from pyspark.sql.functions import sum,avg,max,count

dfVenta.join(dfCliente, dfVenta.IdCliente == dfCliente.IdCliente, "inner") \
    .select(dfCliente.Nombre_y_Apellido, dfVenta.Precio, dfVenta.Cantidad) \
    .groupBy(dfCliente.Nombre_y_Apellido,) \
    .agg(sum(dfVenta.Precio * dfVenta.Cantidad).alias("total")) \
    .orderBy("total", ascending = False) \
    .show(5)


+-----------------+---------+
|Nombre_y_Apellido|    total|
+-----------------+---------+
|Laura Ines Guasch|696645.52|
|    Sergio Artazu|695813.00|
|  Antonio Benitez|621303.84|
|    Irma Esquivel|546859.00|
|         Sin Dato|541618.00|
+-----------------+---------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import sum,avg,max,count
from pyspark.sql.functions import col

dfVenta.alias("venta").join(dfCliente.alias("cliente"), col("venta.IdCliente") == col("cliente.IdCliente"), "inner") \
    .select(col("cliente.Nombre_y_Apellido"), col("venta.Precio"), col("venta.Cantidad")) \
    .groupBy(col("cliente.Nombre_y_Apellido")) \
    .agg(sum(col("venta.Precio") * col("venta.Cantidad")).alias("total")) \
    .orderBy(col("total"), ascending = False) \
    .show(5)